In [2]:
import torch
import torch.nn as nn

In [3]:
decoder = nn.Linear(50,100)

In [4]:
decoder_input = torch.rand(32,100,50)
decoder_output = decoder(decoder_input)
print(decoder_output.shape)

torch.Size([32, 100, 100])


In [8]:
loss_input = decoder_output.reshape(-1, 100)
print(loss_input[0])
print(loss_input.shape)

tensor([-0.5429,  0.1227,  0.0110, -0.4651,  0.7660,  0.0779,  0.2449,  0.3732,
         0.3800, -0.1639,  0.6562,  0.1780,  0.3019,  0.0467,  0.0702,  0.2159,
        -0.1773,  0.0942,  0.3221,  0.4792,  0.1777, -0.1560, -0.6616, -0.1838,
        -0.2362, -0.1038,  0.0331,  0.0358,  0.1601,  0.5581, -0.1937, -0.2368,
        -0.5671, -0.2868, -0.0548,  0.0942,  0.3078, -0.5778, -0.4113, -0.3674,
        -0.4079, -0.0991, -0.0970, -0.7989,  0.4702,  0.1936,  0.1360, -0.0653,
         0.1467,  0.1137,  0.2418,  0.2629,  0.0456,  0.8190, -0.0350,  0.0391,
         0.3904, -0.0106, -0.0492,  0.5394, -0.1494,  0.2481, -0.3660, -0.1801,
        -0.0498, -0.1198, -0.3664,  0.5216, -0.0057,  0.1634,  0.0130, -0.1429,
         0.3137, -0.2151, -0.0798,  0.1564, -0.1899,  0.3421,  0.1557, -0.1384,
        -0.2376, -0.1527,  0.1235,  0.3306, -0.0356, -0.1805, -0.0021,  0.0671,
         0.0292,  0.2990, -0.0171, -0.0305,  0.6222,  0.2481, -0.3432,  0.1940,
         0.2331,  0.1524,  0.1992,  0.29

In [10]:
def get_bucket_limits(num_outputs:int, full_range:tuple=None, ys:torch.Tensor=None):
    assert (ys is not None) or (full_range is not None)
    if ys is not None:
        ys = ys.flatten()
        if len(ys) % num_outputs: ys = ys[:-(len(ys) % num_outputs)]
        print(f'Using {len(ys)} y evals to estimate {num_outputs} buckets. Cut off the last {len(ys) % num_outputs} ys.')
        ys_per_bucket = len(ys) // num_outputs
        if full_range is None:
            full_range = (ys.min(), ys.max())
        else:
            assert full_range[0] <= ys.min() and full_range[1] >= ys.max()
            full_range = torch.tensor(full_range)
        ys_sorted, ys_order = ys.sort(0)
        bucket_limits = (ys_sorted[ys_per_bucket-1::ys_per_bucket][:-1]+ys_sorted[ys_per_bucket::ys_per_bucket])/2
        print(full_range)
        bucket_limits = torch.cat([full_range[0].unsqueeze(0), bucket_limits, full_range[1].unsqueeze(0)],0)

    else:
        class_width = (full_range[1] - full_range[0]) / num_outputs
        bucket_limits = torch.cat([full_range[0] + torch.arange(num_outputs).float()*class_width, torch.tensor(full_range[1]).unsqueeze(0)], 0)

    assert len(bucket_limits) - 1 == num_outputs and full_range[0] == bucket_limits[0] and full_range[-1] == bucket_limits[-1]
    return bucket_limits

In [11]:
borders=get_bucket_limits(100, full_range=(-5,5))
print(borders)

tensor([-5.0000, -4.9000, -4.8000, -4.7000, -4.6000, -4.5000, -4.4000, -4.3000,
        -4.2000, -4.1000, -4.0000, -3.9000, -3.8000, -3.7000, -3.6000, -3.5000,
        -3.4000, -3.3000, -3.2000, -3.1000, -3.0000, -2.9000, -2.8000, -2.7000,
        -2.6000, -2.5000, -2.4000, -2.3000, -2.2000, -2.1000, -2.0000, -1.9000,
        -1.8000, -1.7000, -1.6000, -1.5000, -1.4000, -1.3000, -1.2000, -1.1000,
        -1.0000, -0.9000, -0.8000, -0.7000, -0.6000, -0.5000, -0.4000, -0.3000,
        -0.2000, -0.1000,  0.0000,  0.1000,  0.2000,  0.3000,  0.4000,  0.5000,
         0.6000,  0.7000,  0.8000,  0.9000,  1.0000,  1.1000,  1.2000,  1.3000,
         1.4000,  1.5000,  1.6000,  1.7000,  1.8000,  1.9000,  2.0000,  2.1000,
         2.2000,  2.3000,  2.4000,  2.5000,  2.6000,  2.7000,  2.8000,  2.9000,
         3.0000,  3.1000,  3.2000,  3.3000,  3.4000,  3.5000,  3.6000,  3.7000,
         3.8000,  3.9000,  4.0000,  4.1000,  4.2000,  4.3000,  4.4000,  4.5000,
         4.6000,  4.7000,  4.8000,  4.90

In [59]:
bucket_log_probs = torch.log_softmax(loss_input, -1)
print(bucket_log_probs[0])
print(bucket_log_probs.shape)

bucket_widths = borders[1:] - borders[:-1]
scaled_bucket_log_probs = bucket_log_probs - torch.log(bucket_widths)
print(scaled_bucket_log_probs[0])
print(scaled_bucket_log_probs.shape)

tensor([-5.2321, -4.5665, -4.6782, -5.1543, -3.9232, -4.6113, -4.4443, -4.3160,
        -4.3092, -4.8532, -4.0330, -4.5112, -4.3873, -4.6425, -4.6190, -4.4733,
        -4.8665, -4.5950, -4.3671, -4.2100, -4.5115, -4.8452, -5.3508, -4.8730,
        -4.9254, -4.7930, -4.6561, -4.6535, -4.5291, -4.1311, -4.8829, -4.9260,
        -5.2563, -4.9760, -4.7440, -4.5950, -4.3814, -5.2670, -5.1005, -5.0566,
        -5.0971, -4.7883, -4.7862, -5.4881, -4.2190, -4.4956, -4.5532, -4.7545,
        -4.5425, -4.5755, -4.4474, -4.4263, -4.6436, -3.8702, -4.7242, -4.6501,
        -4.2988, -4.6998, -4.7384, -4.1498, -4.8386, -4.4411, -5.0552, -4.8693,
        -4.7391, -4.8090, -5.0556, -4.1676, -4.6949, -4.5258, -4.6762, -4.8321,
        -4.3755, -4.9043, -4.7690, -4.5328, -4.8791, -4.3471, -4.5335, -4.8276,
        -4.9268, -4.8420, -4.5658, -4.3586, -4.7248, -4.8697, -4.6913, -4.6221,
        -4.6600, -4.3902, -4.7063, -4.7197, -4.0670, -4.4411, -5.0324, -4.4952,
        -4.4561, -4.5368, -4.4900, -4.39

In [15]:
def map_to_bucket_idx(borders, y):
    target_sample = torch.searchsorted(borders, y) - 1
    target_sample[y == borders[0]] = 0
    target_sample[y == borders[-1]] = 100 - 1
    return target_sample

In [72]:
targets = torch.rand(32,100,1).flatten()
print(targets.shape)
target_sample = map_to_bucket_idx(borders, targets)
print(target_sample.shape)
print(target_sample.unsqueeze(-1).shape)
print(target_sample[0])
print(-scaled_bucket_log_probs[0,target_sample[0]])
print(target_sample[-2])
print(-scaled_bucket_log_probs[-2,target_sample[-2]])

torch.Size([3200])
torch.Size([3200])
torch.Size([3200, 1])
tensor(56)
tensor(1.9963, grad_fn=<NegBackward0>)
tensor(54)
tensor(2.6940, grad_fn=<NegBackward0>)


In [73]:
losses = -scaled_bucket_log_probs.gather(-1,target_sample.unsqueeze(-1)).squeeze(-1)
print(losses)
print(losses.size())


tensor([1.9963, 2.0241, 1.8783,  ..., 1.7400, 2.6940, 2.2911],
       grad_fn=<NegBackward0>)
torch.Size([3200])


In [52]:
print(losses.shape)
losses = losses.view(*decoder_output.shape[0:2]).squeeze(-1)
print(losses.shape)
loss = losses.mean()
print(loss.shape)

torch.Size([3200])
torch.Size([32, 100])
torch.Size([])
